In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.linear_model import SGDRegressor


In [2]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## redundant
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [3]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [4]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

## PREPROCESSING

In [5]:
# df_train.columns[df_train.isnull().any()]

In [6]:
# sns.heatmap(df_train.isnull(),yticklabels=False,cbar=False,cmap='viridis')

In [7]:
# plt.figure(figsize=(30,30))
# sns.heatmap(pd.concat([df_train,output_col],axis=1).loc[:,list(pd.concat([df_train,output_col],axis=1).select_dtypes(include= np.number).columns)].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [8]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [9]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(30,30)
# cols=list(df_train.select_dtypes(include= np.number).columns)
# for ax, col in zip(axes, cols):
#     print()
#     sns.distplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [10]:
def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [11]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [12]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


In [13]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [14]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [15]:
most_correlated=dict()
for col1 in continuous_cols:
    for col2 in continuous_cols:
        if col1!=col2 and abs(df_train[col1].corr(df_train[col2],method='spearman'))>=0.80:
            if (col1 not in most_correlated) and (col2 not in most_correlated):
                most_correlated[col1]=col2

In [16]:
df_train.drop(columns=list(most_correlated),axis=1,inplace=True)

In [17]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [18]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [19]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [20]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [21]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

Deleting Outliers.

Here we are deleting outliers using Inter Quartile Range Strategy. Since for a column ouliers are in a specifc row and for different columns the oulier contained rows are different and if we delete those specific row then that row in output label should also be deleted thats why we are first adding label into our features after that we will separate the label.

In [22]:
df_train=pd.concat([output_col,df_train],axis=1) # since rows will be deleted here

In [23]:
# Outlier delete

for col in continuous_cols:
    q1=df_train[col].quantile(0.25)
    q3=df_train[col].quantile(0.75)
    iqr=q3-q1
    l=q1-1.5*iqr
    h=q3+1.5*iqr
    print("[",l,",",h,"]")
    df_train = df_train[(df_train[col] <= h)] 
    df_train = df_train[(df_train[col] >=l)] 

[ -55.0 , 145.0 ]
[ 30.0 , 110.0 ]
[ 2440.0 , 16936.0 ]
[ 2.0 , 10.0 ]
[ 3.5 , 7.5 ]
[ 1909.0 , 2061.0 ]
[ -255.75 , 426.25 ]
[ -1060.5 , 1767.5 ]
[ 0.0 , 0.0 ]
[ -579.875 , 1725.125 ]
[ 199.0 , 2037.0 ]
[ -1092.375 , 1820.625 ]
[ 0.0 , 0.0 ]
[ -1.5 , 2.5 ]
[ 0.0 , 0.0 ]
[ -0.5 , 3.5 ]
[ -1.5 , 2.5 ]
[ 0.5 , 4.5 ]
[ 1.0 , 1.0 ]
[ 2.0 , 10.0 ]
[ -1.5 , 2.5 ]
[ 1903.0 , 2063.0 ]
[ -19.0 , 933.0 ]
[ -239.625 , 399.375 ]
[ -97.5 , 162.5 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.0 , 0.0 ]
[ 0.5 , 12.5 ]
[ 2004.0 , 2012.0 ]


In [24]:

output_col=df_train['SalePrice']
df_train.drop("SalePrice",axis=1,inplace=True)

In [25]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
6,20,RL,75.0,10084,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,57,0,0,0,0,0,8,2007,WD,Normal
10,20,RL,70.0,11200,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,RL,63.0,8500,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,60,0,0,0,0,0,11,2007,WD,Normal
1448,50,RL,70.0,11767,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,24,0,0,0,0,0,5,2007,WD,Normal
1451,20,RL,78.0,9262,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,36,0,0,0,0,0,5,2009,New,Partial
1454,20,FV,62.0,7500,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,113,0,0,0,0,0,10,2009,WD,Normal


Here we are encoding our categorical columns using 'one_hot_encoding' as a strategy. We will be experimenting other strategies later.

In [26]:

def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [28]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="onehotencoder")

In [29]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,60,65.0,8450,7,5,2003,196.0,706,0,150,...,0,0,0,1,0,0,0,0,1,0
2,60,68.0,11250,7,5,2002,162.0,486,0,434,...,0,0,0,1,0,0,0,0,1,0
4,60,84.0,14260,8,5,2000,350.0,655,0,490,...,0,0,0,1,0,0,0,0,1,0
6,20,75.0,10084,8,5,2005,186.0,1369,0,317,...,0,0,0,1,0,0,0,0,1,0
10,20,70.0,11200,5,5,1965,0.0,906,0,134,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,20,63.0,8500,7,5,2004,106.0,0,0,1422,...,0,0,0,1,0,0,0,0,1,0
1448,50,70.0,11767,4,7,2000,0.0,0,0,560,...,0,0,0,1,0,0,0,0,1,0
1451,20,78.0,9262,8,5,2009,194.0,0,0,1573,...,0,1,0,0,0,0,0,0,0,1
1454,20,62.0,7500,7,5,2005,0.0,410,0,811,...,0,0,0,1,0,0,0,0,1,0


In [30]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

Now we will be scaling our columns using Min Max Scaler which will make our value in the range 0 to 1.Scaling is done due to following resons :
1. To make the gradient descent converge fast so to reduce the error as much as possible
2. To make all features on a specific scale.

We do scaling mainly for the distance based algorithms such as KNN,K-Means , SVM and so on. Since it helps them to reduce the error using radient very quickly. But for tree based algorithms we didnt need sacling since their accuracy is based on the split so even though the values are scaled the splitting won't be affected.And since we are using both types of algorithms we will us


In [31]:
scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),index=df_train.index, columns=df_train.columns)
df_train


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0.4,0.4375,0.400706,0.625,0.333333,0.883333,0.463357,0.436072,0.0,0.087771,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.4,0.4750,0.602391,0.625,0.333333,0.866667,0.382979,0.300185,0.0,0.253950,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.4,0.6750,0.819203,0.750,0.333333,0.833333,0.827423,0.404571,0.0,0.286717,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,0.5625,0.518404,0.750,0.333333,0.916667,0.439716,0.845584,0.0,0.185489,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
10,0.0,0.5000,0.598790,0.375,0.333333,0.250000,0.000000,0.559605,0.0,0.078408,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1444,0.0,0.4125,0.404307,0.625,0.333333,0.900000,0.250591,0.000000,0.0,0.832066,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1448,0.3,0.5000,0.639631,0.250,1.000000,0.833333,0.000000,0.000000,0.0,0.327677,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1451,0.0,0.6000,0.459195,0.750,0.333333,0.983333,0.458629,0.000000,0.0,0.920421,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1454,0.0,0.4000,0.332277,0.625,0.333333,0.916667,0.000000,0.253243,0.0,0.474547,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [32]:
from sklearn.metrics import r2_score

In [37]:
X=df_train
Y=output_col

In [38]:
avg_accuracies={}
highest_accuracies={}
algorithms_tested=["LinearRegression()","RandomForestRegressor()","SVR()","KNeighborsRegressor()","SGDRegressor()","DecisionTreeRegressor()","ElasticNet()","Lasso()","Ridge()"]
for al in algorithms_tested:
    avg_accuracies[al]=0
    highest_accuracies[al]=float('-inf')

In [39]:
def algo_testing(algo:str,test_size:float,X,Y,evaluation_metric:str,random_state,shuffle):
    model=eval(algo)
    evaluation_metric=eval(evaluation_metric)
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=random_state,shuffle=shuffle)
   
    model.fit(X_train,Y_train)
    predicted=model.predict(X_test)
    
    r2=evaluation_metric(Y_test, predicted)
    print(f"Testing r2_score of {model} is ",r2)
    print(f"Training r2_score of {model} is ", model.score(X_train,Y_train))
    if highest_accuracies[algo]<r2:
        
        highest_accuracies[algo]=r2
    avg_accuracies[algo]+=r2


In [42]:
for i in range(50):
    algo_testing(algo="LinearRegression()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["LinearRegression()"]/=50



Testing r2_score of LinearRegression() is  -4.989267423077935e+18
Training r2_score of LinearRegression() is  0.9634176282092857
Testing r2_score of LinearRegression() is  -7.914833594608799e+20
Training r2_score of LinearRegression() is  0.9588237493465864
Testing r2_score of LinearRegression() is  -3.5884785466180087e+21
Training r2_score of LinearRegression() is  0.9572176127283847
Testing r2_score of LinearRegression() is  -9.00905499480437e+19
Training r2_score of LinearRegression() is  0.9616302776565767
Testing r2_score of LinearRegression() is  -1.6241119812823958e+19
Training r2_score of LinearRegression() is  0.9627825315654909
Testing r2_score of LinearRegression() is  -1.3864468983129076e+19
Training r2_score of LinearRegression() is  0.9638323338759022
Testing r2_score of LinearRegression() is  -2.609394476302644e+17
Training r2_score of LinearRegression() is  0.9605065404547755
Testing r2_score of LinearRegression() is  -3.554585790993539e+20
Training r2_score of LinearRe

In [41]:

for i in range(50):
    algo_testing(algo="RandomForestRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["RandomForestRegressor()"]/=50

for i in range(50):

    algo_testing(algo="SVR()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SVR()"]/=50

for i in range(50):

    algo_testing(algo="KNeighborsRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["KNeighborsRegressor()"]/=50

for i in range(50):

    algo_testing(algo="SGDRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["SGDRegressor()"]/=50

for i in range(50):

    algo_testing(algo="DecisionTreeRegressor()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["DecisionTreeRegressor()"]/=50

for i in range(50):

    algo_testing(algo="ElasticNet()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["ElasticNet()"]/=50

for i in range(50):

    algo_testing(algo="Lasso()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Lasso()"]/=50

for i in range(50):

    algo_testing(algo="Ridge()",test_size=0.2,X=X,Y=Y,evaluation_metric="r2_score",random_state=None,shuffle=True)
avg_accuracies["Ridge()"]/=50

Testing r2_score of RandomForestRegressor() is  0.8621047937140652
Training r2_score of RandomForestRegressor() is  0.9832777875775764
Testing r2_score of RandomForestRegressor() is  0.7546481022947797
Training r2_score of RandomForestRegressor() is  0.9815929089591211
Testing r2_score of RandomForestRegressor() is  0.8775623266169511
Training r2_score of RandomForestRegressor() is  0.9807355531345958
Testing r2_score of RandomForestRegressor() is  0.9057096766923298
Training r2_score of RandomForestRegressor() is  0.9800679008630854
Testing r2_score of RandomForestRegressor() is  0.8896119385699892
Training r2_score of RandomForestRegressor() is  0.9806587647687046
Testing r2_score of RandomForestRegressor() is  0.87702349637391
Training r2_score of RandomForestRegressor() is  0.9802518252980459
Testing r2_score of RandomForestRegressor() is  0.875068763658562
Training r2_score of RandomForestRegressor() is  0.9814790297234179
Testing r2_score of RandomForestRegressor() is  0.83184319

Testing r2_score of SVR() is  -0.0004895244590898962
Training r2_score of SVR() is  -0.008752166592171928
Testing r2_score of SVR() is  -0.004917435214374599
Training r2_score of SVR() is  -0.008997005332301189
Testing r2_score of SVR() is  -0.006606670566495554
Training r2_score of SVR() is  -0.008889398140030869
Testing r2_score of SVR() is  -0.0014030116086389821
Training r2_score of SVR() is  -0.012229073802211943
Testing r2_score of SVR() is  -0.014621572809854433
Training r2_score of SVR() is  -0.007274785690992713
Testing r2_score of SVR() is  -0.010982277679532748
Training r2_score of SVR() is  -0.006975348663819991
Testing r2_score of SVR() is  0.00046508022712821173
Training r2_score of SVR() is  -0.012098248545216794
Testing r2_score of SVR() is  -0.004685795034176987
Training r2_score of SVR() is  -0.004363129514113773
Testing r2_score of SVR() is  -0.003928623742005755
Training r2_score of SVR() is  -0.00883935684779602
Testing r2_score of SVR() is  -0.023926385908193515
T

Testing r2_score of KNeighborsRegressor() is  0.7447075944156745
Training r2_score of KNeighborsRegressor() is  0.8374507943124603
Testing r2_score of KNeighborsRegressor() is  0.7542811326968355
Training r2_score of KNeighborsRegressor() is  0.8394463990484244
Testing r2_score of KNeighborsRegressor() is  0.7507014458605412
Training r2_score of KNeighborsRegressor() is  0.8372662996773144
Testing r2_score of KNeighborsRegressor() is  0.8327314769529588
Training r2_score of KNeighborsRegressor() is  0.8334249202095246
Testing r2_score of KNeighborsRegressor() is  0.5964406987555158
Training r2_score of KNeighborsRegressor() is  0.8520887290964638
Testing r2_score of KNeighborsRegressor() is  0.770009223971084
Training r2_score of KNeighborsRegressor() is  0.8488229167022658
Testing r2_score of KNeighborsRegressor() is  0.807145219694532
Training r2_score of KNeighborsRegressor() is  0.8359133519259015
Testing r2_score of KNeighborsRegressor() is  0.7171910818898803
Training r2_score of

Testing r2_score of DecisionTreeRegressor() is  0.7844827908801184
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7684372552024681
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7719535105623094
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6295635260270445
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.7326735466687969
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6954797321891725
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.6715896787129634
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor() is  0.765870741320323
Training r2_score of DecisionTreeRegressor() is  1.0
Testing r2_score of DecisionTreeRegressor

Training r2_score of ElasticNet() is  0.7325851767620919
Testing r2_score of ElasticNet() is  0.7238365006091736
Training r2_score of ElasticNet() is  0.7309486918707722
Testing r2_score of ElasticNet() is  0.6329815268411652
Training r2_score of ElasticNet() is  0.75947399706109
Testing r2_score of ElasticNet() is  0.7445514083744595
Training r2_score of ElasticNet() is  0.7323923576340672
Testing r2_score of ElasticNet() is  0.7163750884545297
Training r2_score of ElasticNet() is  0.7354474499469107
Testing r2_score of ElasticNet() is  0.7297279779106778
Training r2_score of ElasticNet() is  0.7463548824057946
Testing r2_score of ElasticNet() is  0.6979821083002687
Training r2_score of ElasticNet() is  0.740876265748395
Testing r2_score of ElasticNet() is  0.733566187243116
Training r2_score of ElasticNet() is  0.7305040419987517
Testing r2_score of ElasticNet() is  0.7012185084078773
Training r2_score of ElasticNet() is  0.7310151738346424
Testing r2_score of ElasticNet() is  0.7091

...

Testing r2_score of Lasso() is  0.8849186427371101
Training r2_score of Lasso() is  0.9631496542453886
Testing r2_score of Lasso() is  0.9065669842965194
Training r2_score of Lasso() is  0.9613809525433268


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.874e+09, tolerance: 1.869e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.561e+08, tolerance: 1.807e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

Testing r2_score of Lasso() is  0.8666500063153949
Training r2_score of Lasso() is  0.9655817581999081
Testing r2_score of Lasso() is  0.9002485732498972
Training r2_score of Lasso() is  0.9615887406862469
Testing r2_score of Lasso() is  0.8715571300351856
Training r2_score of Lasso() is  0.964793469793384
Testing r2_score of Lasso() is  0.8738546467000228
Training r2_score of Lasso() is  0.9644861486066846
Testing r2_score of Lasso() is  0.8959552306056547


C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.580e+09, tolerance: 1.771e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.510e+10, tolerance: 1.860e+08
  model = cd_fast.enet_coordinate_descent(
C:\Users\mritu\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

Training r2_score of Lasso() is  0.961603825103987
Testing r2_score of Lasso() is  0.8877314605484592
Training r2_score of Lasso() is  0.9640596628493256
Testing r2_score of Ridge() is  0.8978743010589605
Training r2_score of Ridge() is  0.9567744288590871
Testing r2_score of Ridge() is  0.9165617248613641
Training r2_score of Ridge() is  0.9534284792709963
Testing r2_score of Ridge() is  0.8925083173894102
Training r2_score of Ridge() is  0.9551257862955846
Testing r2_score of Ridge() is  0.886725574199873
Training r2_score of Ridge() is  0.9600312590022109
Testing r2_score of Ridge() is  0.8924682761581537
Training r2_score of Ridge() is  0.9590796801726158
Testing r2_score of Ridge() is  0.8861689403970496
Training r2_score of Ridge() is  0.9595273924285311
Testing r2_score of Ridge() is  0.8954364548265736
Training r2_score of Ridge() is  0.9540611903407784
Testing r2_score of Ridge() is  0.9037122345158695
Training r2_score of Ridge() is  0.954025511100891
Testing r2_score of Ridg

In [43]:
avg_accuracies

{'LinearRegression()': -1.0095714549029022e+21,
 'RandomForestRegressor()': 0.8620294805144093,
 'SVR()': -0.014845902475678636,
 'KNeighborsRegressor()': 0.7410189642004583,
 'SGDRegressor()': 0.8945534072220188,
 'DecisionTreeRegressor()': 0.7071279889235405,
 'ElasticNet()': 0.7169194597278549,
 'Lasso()': 0.8874557733773425,
 'Ridge()': 0.9060065540575196}

In [44]:
highest_accuracies

{'LinearRegression()': -2.1470695203404444e+16,
 'RandomForestRegressor()': 0.9156020119478879,
 'SVR()': 0.0005025116939700869,
 'KNeighborsRegressor()': 0.8327314769529588,
 'SGDRegressor()': 0.935397569643038,
 'DecisionTreeRegressor()': 0.83161366714345,
 'ElasticNet()': 0.7733999635919844,
 'Lasso()': 0.9246241541359049,
 'Ridge()': 0.9476369041296083}